# Solve a model on a remote machine

In this example, we explain how linopy can handle model optimization on remote machines. 
What you need in order to run the example:
* a running installation of paramiko on your local machine (use `pip install paramiko` for example)
* a remote server with an working installation of linopy, e.g. in a `conda` environment.
* a working ssh access to that machine 


The basic idea that the workflow follows consists of the following steps most of which linopy takes over for you:

1. define a model on the local machine
2. save the model on the remote machine
3. load, solve and write out the model, all on the remote machine
4. copy the solved model to the local machine
5. load the solved model on the local machine

Therefore the initilization process happens locally and the solving remotely. 

## Create a model

First we are going to build the optimization model we want to solve in our local process.

In [ ]:
from linopy import Model
from numpy import arange

N = 10
m = Model()
coords = [arange(N), arange(N)]
x = m.add_variables(coords=coords, name='x')
y = m.add_variables(coords=coords, name='y')
m.add_constraints(x - y >= arange(N))
m.add_constraints(x + y >= 0)
m.add_objective((2 * x + y).sum())
m

## Initialize SSH connection

Now we have to set up the SSH connection. The SSH connection is handled by the `RemoteHandler` class in of the `linopy.remote` module. This is strongly relying on the `paramiko` package. When initializing, you have two options:

1. Pass the standard arguments `host`, `username`. If the SSH keys are stored in a default location, the keys are autodetected and the `RemoteHandler` does not require the `password` argument. Otherwise you also have to pass the password.
2. Pass a working `paramiko.SSHClient` as `client`. This enables you to set up the SSH connection by others means supported by `paramiko`. 

In the following we use the first option.

In [ ]:
from linopy import RemoteHandler

host = "your.host.de"
username = "username"

handler = RemoteHandler(host, username=username)

## Optionally: Activate a conda environment on the remote 

The `RemoteHandler` keeps an interactive shell in the background. You can execute any code in order to prepare the solving process (install linopy, activate an environment). 

Assuming you have a conda environment `linopy-env` that contains the `linopy` package with dependencies, you can run 

In [ ]:
handler.execute("conda activate linopy-env")

## Solve the model on remote

Now the only thing you have to do is to pass the `RemoteHandler` as an argument to the `solve` function. Other keyword arguments like `solver_name` and solver options are propagated to the remote machine. 

In [ ]:
m.solve(remote=handler)

In [ ]:
m.solution